In [133]:
import numpy as np
import pandas as pd
import subprocess
import requests
from time import sleep

In [134]:
def policy(pod, latency):
    return pod + latency

In [135]:
def generate_intervals(initial, interval_size):
    intervals = []
    intervals.append((0, initial))
    for x in range(interval_size - 1):
        initial_interval_value = intervals[-1][1]
        interval_value = intervals[0][1]
        # To use double value
        # last_interval_value = initial_interval_value * 2 
        last_interval_value = initial_interval_value + interval_value
        intervals.append((initial_interval_value + 1, last_interval_value))

    return intervals

In [136]:
# '''
# build_table(3, 5, 2000, 5)


# pod initial_latency	end_latency	      3     4	    5
#   3               0	       2000    2003  2004	 2005
#   4               0	       2000    2004  2005	 2006
#   5               0	       2000    2005  2006	 2007
#   3            2001	       4000    4003  4004	 4005
#   4            2001	       4000    4004  4005	 4006
#   5            2001	       4000    4005  4006	 4007
#   3            4001	       8000    8003  8004	 8005
#   4            4001	       8000    8004  8005	 8006
#   5            4001	       8000    8005  8006	 8007
#   3            8001	      16000   16003 16004	16005
#   4            8001	      16000   16004 16005	16006
#   5            8001	      16000   16005 16006	16007
#   3           16001	      32000   32003 32004	32005
#   4           16001	      32000   32004 32005	32006
#   5           16001	      32000   32005 32006	32007
# '''
# def build_table(min_pods, max_pods, initial_lat, interval_size):
#     intervals = generate_intervals(initial_lat, interval_size)

#     table = []

#     for y in intervals:
#         for x in range(min_pods, max_pods + 1):
#             options = np.zeros(max_pods + 1 - min_pods)
#             options = [policy(z + x, y[1]) for z in range(len(options))]
            
#             table.append([x, y[0], y[1]] + list(options))

#     labels = ['pod', 'initial_latency', 'end_latency']
#     actions = list(np.arange(min_pods, max_pods + 1).astype(np.str_))

#     return pd.DataFrame(table, columns=labels+actions)

In [137]:
'''
build_table(3, 5, 2000, 5)


pod initial_latency	end_latency	      3     4	    5
  3               0	       2000    2003  2004	 2005
  4               0	       2000    2004  2005	 2006
  5               0	       2000    2005  2006	 2007
  3            2001	       4000    4003  4004	 4005
  4            2001	       4000    4004  4005	 4006
  5            2001	       4000    4005  4006	 4007
  3            4001	       8000    8003  8004	 8005
  4            4001	       8000    8004  8005	 8006
  5            4001	       8000    8005  8006	 8007
  3            8001	      16000   16003 16004	16005
  4            8001	      16000   16004 16005	16006
  5            8001	      16000   16005 16006	16007
  3           16001	      32000   32003 32004	32005
  4           16001	      32000   32004 32005	32006
  5           16001	      32000   32005 32006	32007
'''
def build_table(min_pods, max_pods, initial_lat, interval_lat_size, initial_req, interval_req_size):
    requests_intervals = generate_intervals(initial_req, interval_req_size)

    table = []

    for request_interval in requests_intervals:
        latency_intervals = generate_intervals(initial_lat, interval_lat_size)
        for latency_interval in latency_intervals:
            for pod in range(min_pods, max_pods + 1):
                options = np.zeros(max_pods + 1 - min_pods)
                options = [0 for opt in range(len(options))]

                table.append([pod, latency_interval[0], latency_interval[1], request_interval[0], request_interval[1]] + list(options))

    labels = ['pod', 'initial_latency', 'end_latency', 'initial_request', 'end_request']
    actions = list(np.arange(min_pods, max_pods + 1).astype(np.str_))

    return pd.DataFrame(table, columns=labels+actions)

In [138]:
'''
find_option(table, 5, 3000)

   3	    4	    5
2003	 2004	 2005
2004	 2005	 2006
2005	 2006	 2007
'''
def find_options(table, pod, latency, request):
    return table[(table['pod'] == pod) & (table['initial_latency'] <= latency) & (table['end_latency'] >= latency) & (table['initial_request'] <= request) & (table['end_request'] >= request)].iloc[:,5::]

In [139]:
'''
find_best_action(table, 5, 3000)

3
'''
def find_best_action(table, pod, latency, request):
    options = find_options(table, pod, latency, request)
    
    return int(np.max(options).head(1).index.item())

In [140]:
def update_action_result(table, pod, latency, request, action, result):
        table.loc[(table['pod'] == pod) & (table['initial_latency'] <= latency) & (table['end_latency'] >= latency) & (table['initial_request'] <= request) & (table['end_request'] >= request), str(action)] = result
        print("updating pod", pod, " latency ", latency, " request ", request, " result ", result)
        return table

In [141]:
def get_ip():
    # bashCommand = "kubectl get pods --field-selector=status.phase=Running"
    
    error = True
    output = None
    
    while(error == True):
        process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE)
        output, error = process.communicate()
    
    return output.decode("utf-8").split('\n')[1:][:-1].pop().split('   ')[3]

In [142]:
'''
return ms latency
'''
def get_latency():
    query = 'rate(http_server_request_duration_seconds_sum{path="/test"}[5m])/rate(http_server_request_duration_seconds_count{path="/test"}[5m])'
    result = None

    while(True):
        response = requests.get("http://localhost:9090/api/v1/query?query={query}".format(query = query))
        if response.json()['status'] == 'success':
            result = response.json()['data']['result']
            if result == []:
                result = float(0)
            else:
                result = response.json()['data']['result'][0]['value'][1]
                if result == 'NaN': result = float(0)
            break
        sleep(5)
    
    return float(result) * 1000

In [143]:
'''
return requests count
'''
def get_request_by_second():
    query = 'increase(http_server_requests_total{path="/test"}[5m])'
    result = None

    while(True):
        response = requests.get("http://localhost:9090/api/v1/query?query={query}".format(query = query))
        if response.json()['status'] == 'success':
            result = response.json()['data']['result']
            if result == []:
                result = float(0)
            else:
                result = response.json()['data']['result'][0]['value'][1]
                if result == 'NaN': result = float(0)
            break
        sleep(5)
    
    return float(result)

In [144]:
def get_pods():
    # TODO comando pega quando esta como Terminating
    bashCommand = "kubectl get pods --field-selector=status.phase=Running"
    
    error = True
    output = None
    
    while(error == True):
        process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE)
        output, error = process.communicate()
    
    return len(output.decode("utf-8").split('\n')[1:][:-1])  
    

In [145]:
def set_pods(new_pods):
    bashCommand = "kubectl scale deployment.v1.apps/phpa-web-app-deployment --replicas={pods}".format(pods = new_pods)

    error = True
    output = None

    while(error == True):
        process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE)
        output, error = process.communicate()

In [146]:
def run(epochs, table, min_pods, max_pods, reward):
    epsilon = 0.5
    ideal_latency = 500
    
    for x in range(epochs):
        latency = get_latency()
        pods = get_pods()
        requests = get_request_by_second()

        print("\n--- Exec ", x)
        print("latency ", latency)
        print("pods ", pods)
        print("requests ", requests)

        if np.random.random() > epsilon:
            action = find_best_action(table, pods, latency, requests)
            print("choose action ", action)
        else:
            action = np.random.randint(min_pods, max_pods + 1)
            print("random ", action)

        set_pods(action)
        sleep(75)

        new_latency = get_latency()

        print("new_latency ", new_latency)
        print("new_result ", new_result)  
        print("requests ", requests)

        reward = get_reward(ideal_latency, pods, action, latency, new_latency)
        print("reward ", reward)
        
        table = update_action_result(table, pods, latency, requests, action, reward)

In [147]:
def is_ideal_latency(ideal_latency, latency):
    return latency <= ideal_latency

In [148]:
def obtained_result(new_value, old_value):
    if new_value == old_value:
        return 'kept'
    elif new_value > old_value:
        return 'increased'
    else:
        return 'decreased'

In [149]:
def reward_within_ideal(pod_state, latency_state, new_latency_is_ideal):
    if not new_latency_is_ideal:
        return -20 # qualquer ação que tomou resultou em levar a latencia para acima do desejado
    
    if pod_state == 'kept': return 10 # não tomou nenhuma ação, mas manteve a latencia ideal
    elif pod_state == 'increased':
        return {
            'kept': 0, # aumentou pods e a latencia se manteve - pode ter aumentado as requisições
            'increased': 10, # aumentou pods e a latencia aumentou - pode ter aumentado as requisições
            'decreased': -20 # ja estava na latencia ideal e aumentou pods desnecessáriamente
        }.get(latency_state)
    else: # pod_state == 'decreased'
        return {
            'kept': 20, # diminiu pods e a latencia se manteve
            'increased': 30, # diminiu pods, latencia aumentou mas segue estando dentro do desejado: ótimo cenário
            'decreased': 20 # diminiu pods, diminiu latencia - pode ter diminuido as requisições
        }.get(latency_state)

In [150]:
def reward_out_ideal(pod_state, latency_state, new_latency_is_ideal):
    if new_latency_is_ideal:
        return 30 # qualquer ação tomada resultou em levar a latencia para dentro do desejado
    
    if pod_state == 'kept':
        return {
            'kept': -20, # não tomou nenhuma ação
            'increased': -20, # não tomou nenhuma ação e aumentou a latencua
            'decreased': -10 # não tomou nenhuma ação mas diminiu a latencia - pode ter diminuido requisições
        }.get(latency_state)
    elif pod_state == 'increased':
        return {
            'kept': 0, # aumentou pods e manteve a latencia - podem ter aumentado as requisições
            'increased': -10, # aumentou pods e aumentou latencia - podem ter aumentado as requisições
            'decreased': 20 # aumentou pods e diminiu a latencia
        }.get(latency_state)
    else: # pod_state == 'decreased'
        return {
            'kept': 0, # diminiu pods e a latencia se manteve
            'increased': -20, # diminiu pods, latencia aumentou
            'decreased': 10 # diminiu pods, diminiu latencia - pode ter diminuido as requisições
        }.get(latency_state)

In [151]:
def get_reward(ideal_latency, old_pods, new_pods, old_latency, new_latency):
    pod_state = obtained_result(new_pods, old_pods)
    latency_state = obtained_result(new_latency, old_latency)

    old_latency_is_ideal = is_ideal_latency(ideal_latency, old_latency)
    new_latency_is_ideal = is_ideal_latency(ideal_latency, new_latency)

    reward = 0
    if old_latency_is_ideal:
        reward = reward_within_ideal(pod_state, latency_state, new_latency_is_ideal)
    else:
        reward = reward_out_ideal(pod_state, latency_state, new_latency_is_ideal)

    return reward

In [155]:
epochs = 10
min_pods = 1
max_pods = 3
initial_latency = 500
interval_size = 4
initial_request = 18
interval_request = 3
reward = 0

epsilon = 0.5

In [156]:
table = build_table(min_pods, max_pods, initial_latency, interval_size, initial_request, interval_request)
table

,pod,initial_latency,end_latency,initial_request,end_request,1,2,3
0,1,0,500,0,18,0,0,0
1,2,0,500,0,18,0,0,0
2,3,0,500,0,18,0,0,0
3,1,501,1000,0,18,0,0,0
4,2,501,1000,0,18,0,0,0
5,3,501,1000,0,18,0,0,0
6,1,1001,1500,0,18,0,0,0
7,2,1001,1500,0,18,0,0,0
8,3,1001,1500,0,18,0,0,0
9,1,1501,2000,0,18,0,0,0


In [157]:
run(epochs, table, min_pods, max_pods, reward)

ConnectionError: HTTPConnectionPool(host='localhost', port=9090): Max retries exceeded with url: /api/v1/query?query=rate(http_server_request_duration_seconds_sum%7Bpath=%22/test%22%7D%5B5m%5D)/rate(http_server_request_duration_seconds_count%7Bpath=%22/test%22%7D%5B5m%5D) (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7ffa857f5a00>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [231]:
table

,pod,initial_latency,end_latency,initial_request,end_request,1,2,3
0,1,0,500,0,50,501.000000,502.000000,503.000000
1,2,0,500,0,50,502.000000,503.000000,357.331735
2,3,0,500,0,50,1958.983628,504.000000,357.331735
3,1,501,1000,0,50,1001.000000,1002.000000,1003.000000
4,2,501,1000,0,50,1002.000000,2001.622764,1004.000000
...,...,...,...,...,...,...,...,...
1075,2,5001,5500,1451,1500,5502.000000,5503.000000,5504.000000
1076,3,5001,5500,1451,1500,5503.000000,5504.000000,5505.000000
1077,1,5501,6000,1451,1500,6001.000000,6002.000000,6003.000000
1078,2,5501,6000,1451,1500,6002.000000,6003.000000,6004.000000


In [24]:
table2 = build_table(min_pods, max_pods, initial_latency, interval_size)
table2

,pod,initial_latency,end_latency,1,2,3
0,1,0,500,501,502,503
1,2,0,500,502,503,504
2,3,0,500,503,504,505
3,1,501,1000,1001,1002,1003
4,2,501,1000,1002,1003,1004
5,3,501,1000,1003,1004,1005
6,1,1001,1500,1501,1502,1503
7,2,1001,1500,1502,1503,1504
8,3,1001,1500,1503,1504,1505
9,1,1501,2000,2001,2002,2003
